
# **Análise de Vendas e Performance Comercial com SQL no Databricks**  

## **Introdução**  
Em um cenário altamente competitivo, empresas buscam constantemente otimizar suas estratégias comerciais para aumentar a eficiência e maximizar os resultados. O uso de sistemas de **Customer Relationship Management (CRM)** tem se tornado essencial para acompanhar interações com clientes, oportunidades de vendas e métricas de desempenho da equipe comercial.  

Este projeto tem como objetivo explorar e analisar um conjunto de dados de CRM, utilizando **SQL no Databricks** para extrair insights valiosos sobre o processo de vendas. Através da análise dos dados, será possível identificar padrões, avaliar o desempenho da equipe comercial e encontrar oportunidades de crescimento para o negócio.   

## **Sobre o Conjunto de Dados**  
O dataset contém informações detalhadas sobre contas de clientes, oportunidades de vendas, atividades comerciais e desempenho da equipe de vendas. Ele é composto por cinco arquivos principais:  

📊 accounts.csv – Informações sobre os clientes, incluindo dados demográficos e firmográficos.
📊 sales_pipeline.csv – Registro das oportunidades de vendas, incluindo estágio, valor e probabilidade de conversão.
📊 products.csv – Lista de produtos e serviços oferecidos pela empresa.
📊 agents.csv – Informações sobre os vendedores e métricas de desempenho.

## **Ferramentas e Tecnologias Utilizadas**  
🛠 **Databricks** – Plataforma baseada em Apache Spark para processamento de dados em larga escala.  
🛠 **SQL** – Linguagem de consulta para análise e manipulação dos dados.  
🛠 **Visualizações no Databricks** – Dashboards interativo para análise e insights.  



## Carregar os arquivos CSV como DataFrames no Databricks

In [0]:
from pyspark.sql import SparkSession

# Criar sessão Spark (Databricks já inicializa automaticamente)
spark = SparkSession.builder.appName("CRM Analysis").getOrCreate()

# Definir caminho base atualizado
base_path = "dbfs:/FileStore/CRM Analytics/"

# Carregar os arquivos CSV como DataFrames
accounts_df = spark.read.csv(base_path + "accounts.csv", header=True, inferSchema=True)
data_dictionary_df = spark.read.csv(base_path + "data_dictionary.csv", header=True, inferSchema=True)
products_df = spark.read.csv(base_path + "products.csv", header=True, inferSchema=True)
sales_pipeline_df = spark.read.csv(base_path + "sales_pipeline.csv", header=True, inferSchema=True)
sales_teams_df = spark.read.csv(base_path + "sales_teams.csv", header=True, inferSchema=True)

# Exibir as primeiras linhas de cada DataFrame para conferir
accounts_df.show(5)
sales_pipeline_df.show(5)


+----------------+---------+----------------+-------+---------+---------------+-------------+
|         account|   sector|year_established|revenue|employees|office_location|subsidiary_of|
+----------------+---------+----------------+-------+---------+---------------+-------------+
|Acme Corporation|technolgy|            1996|1100.04|     2822|  United States|         null|
|      Betasoloin|  medical|            1999| 251.41|      495|  United States|         null|
|        Betatech|  medical|            1986| 647.18|     1185|          Kenya|         null|
|      Bioholding|  medical|            2012| 587.34|     1356|     Philipines|         null|
|         Bioplex|  medical|            1991| 326.82|     1016|  United States|         null|
+----------------+---------+----------------+-------+---------+---------------+-------------+
only showing top 5 rows

+--------------+---------------+--------------+-------+----------+-----------+----------+-----------+
|opportunity_id|    sales_a

## Verificar o esquema das tabelas

In [0]:
accounts_df.printSchema()
sales_pipeline_df.printSchema()
products_df.printSchema()
sales_pipeline_df.printSchema()

root
 |-- account: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- year_established: integer (nullable = true)
 |-- revenue: double (nullable = true)
 |-- employees: integer (nullable = true)
 |-- office_location: string (nullable = true)
 |-- subsidiary_of: string (nullable = true)

root
 |-- opportunity_id: string (nullable = true)
 |-- sales_agent: string (nullable = true)
 |-- product: string (nullable = true)
 |-- account: string (nullable = true)
 |-- deal_stage: string (nullable = true)
 |-- engage_date: date (nullable = true)
 |-- close_date: date (nullable = true)
 |-- close_value: integer (nullable = true)

root
 |-- product: string (nullable = true)
 |-- series: string (nullable = true)
 |-- sales_price: integer (nullable = true)

root
 |-- opportunity_id: string (nullable = true)
 |-- sales_agent: string (nullable = true)
 |-- product: string (nullable = true)
 |-- account: string (nullable = true)
 |-- deal_stage: string (nullable = true)
 |-- engage_da

## Tratamento dos dados

In [0]:
from pyspark.sql.functions import col, sum

# Contar valores nulos por coluna em cada DataFrame
def count_nulls(df, df_name):
    print(f"Valores nulos em {df_name}:")
    df.select([col(c).isNull().cast("int").alias(c) for c in df.columns]).agg(*[sum(col(c)).alias(c) for c in df.columns]).show()

count_nulls(accounts_df, "Accounts")
count_nulls(sales_pipeline_df, "Sales Pipeline")
count_nulls(products_df, "Products")
count_nulls(sales_teams_df, "Sales Teams")



Valores nulos em Accounts:
+-------+------+----------------+-------+---------+---------------+-------------+
|account|sector|year_established|revenue|employees|office_location|subsidiary_of|
+-------+------+----------------+-------+---------+---------------+-------------+
|      0|     0|               0|      0|        0|              0|           70|
+-------+------+----------------+-------+---------+---------------+-------------+

Valores nulos em Sales Pipeline:
+--------------+-----------+-------+-------+----------+-----------+----------+-----------+
|opportunity_id|sales_agent|product|account|deal_stage|engage_date|close_date|close_value|
+--------------+-----------+-------+-------+----------+-----------+----------+-----------+
|             0|          0|      0|   1425|         0|        500|      2089|       2089|
+--------------+-----------+-------+-------+----------+-----------+----------+-----------+

Valores nulos em Products:
+-------+------+-----------+
|product|series|s

In [0]:
# Preencher valores nulos em cada coluna com um valor específico
accounts_df = accounts_df.fillna({
    'subsidiary_of': 'Desconhecido',  # Para coluna 'subsidiary_of'
    'sector': 'Desconhecido',         # Para coluna 'sector', caso necessário
    'year_established': 0,            # Para ano de fundação
    'revenue': 0,                     # Para receita
    'employees': 0,                   # Para número de empregados
    'office_location': 'Desconhecido' # Para local da sede
})

sales_pipeline_df = sales_pipeline_df.fillna({
    'account': 'Desconhecido',        # Para conta
    'engage_date': 'Desconhecido',    # Para data de engajamento
    'close_date': 'Desconhecido',     # Para data de fechamento
    'close_value': 0                  # Para valor de fechamento
})

accounts_df.show()
sales_pipeline_df.show()



+----------------+-------------+----------------+-------+---------+---------------+----------------+
|         account|       sector|year_established|revenue|employees|office_location|   subsidiary_of|
+----------------+-------------+----------------+-------+---------+---------------+----------------+
|Acme Corporation|    technolgy|            1996|1100.04|     2822|  United States|    Desconhecido|
|      Betasoloin|      medical|            1999| 251.41|      495|  United States|    Desconhecido|
|        Betatech|      medical|            1986| 647.18|     1185|          Kenya|    Desconhecido|
|      Bioholding|      medical|            2012| 587.34|     1356|     Philipines|    Desconhecido|
|         Bioplex|      medical|            1991| 326.82|     1016|  United States|    Desconhecido|
|        Blackzim|       retail|            2009| 497.11|     1588|  United States|    Desconhecido|
|   Bluth Company|    technolgy|            1993|1242.32|     3027|  United States|Acme Cor

In [0]:
# Verificando valores duplicatos
dfs = [accounts_df, data_dictionary_df, products_df, sales_pipeline_df, sales_teams_df]
names = ["accounts_df", "data_dictionary_df", "products_df", "sales_pipeline_df", "sales_teams_df"]

# Função para verificar duplicatas em cada DataFrame
for df, name in zip(dfs, names):
    duplicates = df.count() - df.dropDuplicates().count()
    print(f"Duplicatas no DataFrame '{name}': {duplicates}")



Duplicatas no DataFrame 'accounts_df': 0
Duplicatas no DataFrame 'data_dictionary_df': 0
Duplicatas no DataFrame 'products_df': 0
Duplicatas no DataFrame 'sales_pipeline_df': 0
Duplicatas no DataFrame 'sales_teams_df': 0


## Registrar as tabelas como views temporárias

In [0]:
accounts_df.createOrReplaceTempView("accounts")
sales_pipeline_df.createOrReplaceTempView("sales_pipeline")
products_df.createOrReplaceTempView("products")
sales_teams_df.createOrReplaceTempView("sales_teams")

## Análise de CRM

### 1. Quais são os produtos mais vendidos por cada conta?

In [0]:
%sql
WITH SalesSummary AS (
    SELECT
        a.account,              
        p.product,             
        SUM(sp.close_value) AS total_sales,
        RANK() OVER (PARTITION BY a.account ORDER BY SUM(sp.close_value) DESC) AS rank
    FROM accounts AS a
    JOIN sales_pipeline AS sp ON a.account = sp.account 
    JOIN products AS p ON sp.product = p.product          
    GROUP BY a.account, p.product
)
SELECT 
    account, 
    product, 
    total_sales
FROM SalesSummary
WHERE rank = 1
ORDER BY total_sales DESC;


account,product,total_sales
Kan-code,GTX Plus Pro,92402
Cheers,GTK 500,82601
Konex,MG Advanced,74379
Condax,GTX Plus Pro,70345
Goodsilron,GTX Plus Pro,65692
Treequote,GTX Plus Pro,62963
Rangreen,MG Advanced,60111
Lexiqvolax,GTX Plus Pro,56805
dambase,GTX Plus Pro,55656
Xx-holding,GTK 500,55406


Databricks visualization. Run in Databricks to view.

### 2. Qual é o valor total de vendas fechado por cada produto?

In [0]:
%sql
SELECT
    sp.product,                               
    SUM(sp.close_value) AS total_sales_value  
FROM
    sales_pipeline AS sp
JOIN
    products AS p ON sp.product = p.product   
GROUP BY
    sp.product                                
ORDER BY
    total_sales_value DESC;                  

product,total_sales_value
GTX Plus Pro,2629651
MG Advanced,2216387
GTX Plus Basic,705275
GTX Basic,499263
GTK 500,400612
MG Special,43768


Databricks visualization. Run in Databricks to view.

## 3. Quais são os agentes de vendas que mais fecharam vendas (valor total de vendas)?

In [0]:
%sql
SELECT
    st.sales_agent,                           
    SUM(sp.close_value) AS total_sales_value  
FROM
    sales_pipeline sp
JOIN
    sales_teams AS st ON sp.sales_agent = st.sales_agent  
GROUP BY
    st.sales_agent                            
ORDER BY
    total_sales_value DESC;                 

sales_agent,total_sales_value
Darcel Schlecht,1153214
Vicki Laflamme,478396
Kary Hendrixson,454298
Cassey Cress,450489
Donn Cantrell,445860
Reed Clapper,438336
Zane Levy,430068
Corliss Cosme,421036
James Ascencio,413533
Daniell Hammack,364229


Databricks visualization. Run in Databricks to view.

## 4. Qual é o perfil de clientes que geram maior valor de vendas?

In [0]:
%sql
SELECT 
    a.account, 
    a.sector, 
    a.year_established, 
    SUM(sp.close_value) AS total_sales_value
FROM 
    accounts a
JOIN 
    sales_pipeline sp ON a.account = sp.account
GROUP BY 
    a.account, a.sector, a.year_established
ORDER BY 
    total_sales_value DESC

account,sector,year_established,total_sales_value
Kan-code,software,1982,341455
Konex,technolgy,1980,269245
Condax,medical,2017,206410
Cheers,entertainment,1993,198020
Hottechi,technolgy,1997,194957
Goodsilron,marketing,2000,182522
Treequote,telecommunications,1988,176751
Warephase,services,1997,170046
Xx-holding,finance,1993,169357
Isdom,medical,2002,164683


Databricks visualization. Run in Databricks to view.

## 5. Quais são os gargalos no pipeline de vendas e como podemos melhorar o processo

In [0]:
%sql
SELECT 
    sp.deal_stage, 
    COUNT(sp.opportunity_id) AS stage_opportunity_count
FROM 
    sales_pipeline sp
GROUP BY 
    sp.deal_stage
ORDER BY 
    stage_opportunity_count DESC

deal_stage,stage_opportunity_count
Won,4238
Lost,2473
Engaging,1589
Prospecting,500


Databricks visualization. Run in Databricks to view.